In [1]:
import json
import numpy as np
import os
import pandas as pd
import urllib2
from datetime import datetime
import calendar
from keras import applications
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation, Reshape, LeakyReLU
from keras.callbacks import CSVLogger
import tensorflow as tf
from scipy.ndimage import imread
import numpy as np
import random
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
import keras
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
import h5py
from sklearn.preprocessing import MinMaxScaler
import os
import pandas as pd
# import matplotlib
import h5py


Using TensorFlow backend.


In [2]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [4]:
d = datetime.utcnow()
unixtime = calendar.timegm(d.utctimetuple())

unixtime = unixtime /100 *100
past_unixtime = unixtime- 300*300
past_unixtime

1514770800

In [5]:
url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start='+str(past_unixtime)+'&end=9999999999&period=300'
openUrl = urllib2.urlopen(url)
r = openUrl.read()
openUrl.close()
d = json.loads(r.decode())


In [6]:
df = pd.DataFrame(d)
original_columns=[u'close', u'date', u'high', u'low', u'open']
new_columns = ['Close','Timestamp','High','Low','Open']
df = df.loc[:,original_columns]
df.columns = new_columns
df = df.iloc[-256:,:]
df.shape

(256, 5)

In [7]:
datas = df.Close
datas.shape

(256,)

In [8]:
with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    original_datas = hf['original_datas'].value

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(original_datas[:,0].reshape(-1,1))
datas= scaler.transform(datas.reshape(-1,1))

/home/data/khuangaf/miniconda2/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.


In [10]:
datas=datas[None,:,:]
datas.shape

(1, 256, 1)

In [11]:
step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]
epochs = 1
output_size=16
units= 50
second_units=30

In [12]:
model = Sequential()
model.add(LSTM(units=units, activation=None, input_shape=(step_size,nb_features),return_sequences=False))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(output_size))
model.add(LeakyReLU())
model.load_weights('weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-44-0.00004.hdf5')
model.compile(loss='mse', optimizer='adam')

In [13]:
predicted = model.predict(datas)
predicted_inverted = scaler.inverse_transform(predicted)
predicted_inverted

array([[ 13780.57421875,  13559.92089844,  13505.578125  ,  13531.57128906,
         13530.35742188,  13586.72265625,  13495.48632812,  13455.85839844,
         13488.28320312,  13540.39453125,  13371.14746094,  13469.58105469,
         13472.74023438,  13507.29589844,  13599.37011719,  13590.74511719]], dtype=float32)

In [14]:
class MyEncoder(json.JSONEncoder):
    def encode(self, obj):
        if isinstance(obj, float):
            return format(obj, '.2f')
        return json.JSONEncoder.encode(self, obj)

In [26]:
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.2f')
json.dumps([ float(a) for a in list(predicted_inverted.reshape(-1))], cls=MyEncoder )

'[13780.57421875, 13559.9208984375, 13505.578125, 13531.5712890625, 13530.357421875, 13586.72265625, 13495.486328125, 13455.8583984375, 13488.283203125, 13540.39453125, 13371.1474609375, 13469.5810546875, 13472.740234375, 13507.2958984375, 13599.3701171875, 13590.7451171875]'

In [19]:
import simplejson

class PrettyFloat(float):
    def __repr__(self):
        return '%.15g' % self

def pretty_floats(obj):
    if isinstance(obj, float):
        return PrettyFloat(obj)
    elif isinstance(obj, dict):
        return dict((k, pretty_floats(v)) for k, v in obj.items())
    elif isinstance(obj, (list, tuple)):
        return map(pretty_floats, obj)             
    return obj

In [25]:
simplejson.dumps(pretty_floats([ float(np.round(a)) for a in list(predicted_inverted.reshape(-1))]))

'[13781.0, 13560.0, 13506.0, 13532.0, 13530.0, 13587.0, 13495.0, 13456.0, 13488.0, 13540.0, 13371.0, 13470.0, 13473.0, 13507.0, 13599.0, 13591.0]'

In [23]:
[ float(a) for a in list(predicted_inverted.reshape(-1))]

[13781.0,
 13560.0,
 13506.0,
 13532.0,
 13530.0,
 13587.0,
 13495.0,
 13456.0,
 13488.0,
 13540.0,
 13371.0,
 13470.0,
 13473.0,
 13507.0,
 13599.0,
 13591.0]